# CatBoostモデルを用いるシート

In [4]:
from catboost import CatBoostRegressor
from catboost import Pool

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
# import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error
import japanize_matplotlib
from sklearn.model_selection import KFold
import xgboost
import ipywidgets

###訓練データの読み込み
house_age = pd.read_csv('../data/handmade_data/house_age.csv')
area_size = pd.read_csv('../data/handmade_data/area_size.csv')
room_arrange = pd.read_csv('../data/handmade_data/room_arrange.csv')
contract_span = pd.read_csv('../data/handmade_data/contract_span.csv')#欠損値をゼロ埋めした契約期間
contract_span2 = pd.read_csv('../data/handmade_data/contract_span2.csv')#欠損値を欠損値のままにした契約期間
reg_rent = pd.read_csv('../data/handmade_data/reg_rent.csv')
floor_scores = pd.read_csv('../data/handmade_data/floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「所在階」
floor_scores2 = pd.read_csv('../data/handmade_data/floor_scores2.csv')#「所在階」と「全体の階数」が独立している「所在階」
Floor_scores = pd.read_csv('../data/handmade_data/capital_floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「全体の階数」
Floor_scores2 = pd.read_csv('../data/handmade_data/capital_floor_scores2.csv')#「所在階」と「全体の階数」が独立している「全体の階数」
stations = pd.read_csv('../data/handmade_data/stations.csv')
minits = pd.read_csv('../data/handmade_data/minits.csv')
addresses = pd.read_csv('../data/handmade_data/addresses.csv')
room_arrange = pd.read_csv('../data/handmade_data/room_arrange.csv')
buildings = pd.read_csv('../data/handmade_data/buildings.csv')
land_aveprice = pd.read_csv('../data/handmade_data/land_aveprice.csv')
land_aveprice.columns = ['平均地価']
over100m2 = pd.read_csv('../data/handmade_data/over100m2.csv')
bt75and100m2 = pd.read_csv('../data/handmade_data/bt75and100m2.csv')

rent = pd.read_csv('../data/handmade_data/rent.csv')


##テストデータの読み込み
test_house_age = pd.read_csv('../data/test_data/test_house_age.csv')
test_area_size = pd.read_csv('../data/test_data/test_area_size.csv')
test_room_arrange = pd.read_csv('../data/test_data/test_room_arrange.csv')
test_contract_span = pd.read_csv('../data/test_data/test_contract_span.csv')
test_contract_span2 = pd.read_csv('../data/test_data/test_contract_span2.csv')
test_contract_span2.columns = ['契約期間']
test_reg_rent = pd.read_csv('../data/test_data/test_reg_rent.csv')
test_floor_scores = pd.read_csv('../data/test_data/test_floor_scores.csv')
test_floor_scores2 = pd.read_csv('../data/test_data/test_floor_scores2.csv')
test_Floor_scores = pd.read_csv('../data/test_data/test_capital_floor_scores.csv')
test_Floor_scores2 = pd.read_csv('../data/test_data/test_capital_floor_scores2.csv')
test_stations = pd.read_csv('../data/test_data/test_stations.csv')
test_minits = pd.read_csv('../data/test_data/test_minits.csv')
test_addresses = pd.read_csv('../data/test_data/test_addresses.csv')
test_room_arrange = pd.read_csv('../data/test_data/test_room_arrange.csv')
test_buildings = pd.read_csv('../data/test_data/test_buildings.csv')
test_land_aveprice = pd.read_csv('../data/test_data/test_land_aveprice.csv')
test_land_aveprice.columns = ['平均地価']
test_over100m2 = pd.read_csv('../data/test_data/test_over100m2.csv')
test_bt75and100m2 = pd.read_csv('../data/test_data/test_bt75and100m2.csv')


X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, land_aveprice, addresses, room_arrange[["部屋数","L","D","S"]], over100m2, bt75and100m2], axis=1)
y_train = pd.read_csv('../data/handmade_data/rent.csv')
y_train.loc[[5775], '賃料'] = y_train.loc[[5775], '賃料']/10
X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_land_aveprice, test_addresses, test_room_arrange[["部屋数","L","D","S"]], test_over100m2, test_bt75and100m2], axis=1)
category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D','S', '100m^2以上', '75m^2以上100m^2未満']

In [3]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index,:]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train.loc[train_index]
    y_val = y_train.loc[valid_index]

    model = CatBoostRegressor(
        early_stopping_rounds=10,
        learning_rate=0.05, 
        depth=6, 
        loss_function='RMSE', 
        l2_leaf_reg=0.0001
        )

    model.fit(X_tr, y_tr, plot=True)

y_pred = model.predict(X_test)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 71188.9241664	total: 59.2ms	remaining: 59.1s
1:	learn: 68688.4187199	total: 61.5ms	remaining: 30.7s
2:	learn: 66345.6485263	total: 64.1ms	remaining: 21.3s
3:	learn: 64115.5745278	total: 66.4ms	remaining: 16.5s
4:	learn: 62093.7735571	total: 68.4ms	remaining: 13.6s
5:	learn: 60081.0336810	total: 70.6ms	remaining: 11.7s
6:	learn: 58109.6452864	total: 72.6ms	remaining: 10.3s
7:	learn: 56308.0493380	total: 74.5ms	remaining: 9.23s
8:	learn: 54524.4356038	total: 76.7ms	remaining: 8.45s
9:	learn: 52900.2010815	total: 78.9ms	remaining: 7.81s
10:	learn: 51353.2812247	total: 80.8ms	remaining: 7.26s
11:	learn: 49941.4171476	total: 82.7ms	remaining: 6.81s
12:	learn: 48559.7159805	total: 84.8ms	remaining: 6.44s
13:	learn: 47231.5663788	total: 87.3ms	remaining: 6.15s
14:	learn: 45984.1431313	total: 89.5ms	remaining: 5.88s
15:	learn: 44776.6530239	total: 91.5ms	remaining: 5.63s
16:	learn: 43671.3080824	total: 93.7ms	remaining: 5.42s
17:	learn: 42625.7315867	total: 95.7ms	remaining: 5.22s
18

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 70770.7043393	total: 1.93ms	remaining: 1.93s
1:	learn: 68357.7231050	total: 3.77ms	remaining: 1.88s
2:	learn: 66042.8975168	total: 5.37ms	remaining: 1.78s
3:	learn: 63930.6855774	total: 7.26ms	remaining: 1.81s
4:	learn: 61807.2656063	total: 9.38ms	remaining: 1.87s
5:	learn: 59775.4575601	total: 12.2ms	remaining: 2.02s
6:	learn: 57965.5608590	total: 15.1ms	remaining: 2.15s
7:	learn: 56198.4201710	total: 17.8ms	remaining: 2.2s
8:	learn: 54439.8864053	total: 19.8ms	remaining: 2.18s
9:	learn: 52863.9228997	total: 22.2ms	remaining: 2.2s
10:	learn: 51373.3999800	total: 24.4ms	remaining: 2.19s
11:	learn: 49959.0916205	total: 26.5ms	remaining: 2.19s
12:	learn: 48564.0819849	total: 28.8ms	remaining: 2.19s
13:	learn: 47282.5268125	total: 31.6ms	remaining: 2.22s
14:	learn: 46118.2392061	total: 33.9ms	remaining: 2.23s
15:	learn: 44918.0322357	total: 35.9ms	remaining: 2.21s
16:	learn: 43824.3167057	total: 38.3ms	remaining: 2.21s
17:	learn: 42718.6100340	total: 40.5ms	remaining: 2.21s
18:	

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 70787.1802142	total: 2.14ms	remaining: 2.14s
1:	learn: 68331.6600116	total: 3.99ms	remaining: 1.99s
2:	learn: 66066.9384735	total: 5.77ms	remaining: 1.92s
3:	learn: 63892.6359448	total: 7.37ms	remaining: 1.84s
4:	learn: 61742.1115317	total: 9.28ms	remaining: 1.85s
5:	learn: 59771.6486909	total: 11.7ms	remaining: 1.94s
6:	learn: 57928.9466475	total: 14.2ms	remaining: 2.02s
7:	learn: 56142.9819914	total: 17.2ms	remaining: 2.13s
8:	learn: 54430.4989921	total: 19.7ms	remaining: 2.17s
9:	learn: 52845.8554541	total: 22ms	remaining: 2.17s
10:	learn: 51309.0142123	total: 24.3ms	remaining: 2.19s
11:	learn: 49823.8137938	total: 26.5ms	remaining: 2.19s
12:	learn: 48420.1263093	total: 29ms	remaining: 2.2s
13:	learn: 47068.3852480	total: 31.7ms	remaining: 2.23s
14:	learn: 45876.0322324	total: 34.3ms	remaining: 2.25s
15:	learn: 44673.4219682	total: 36.6ms	remaining: 2.25s
16:	learn: 43545.5101006	total: 38.9ms	remaining: 2.25s
17:	learn: 42543.6126550	total: 41.3ms	remaining: 2.25s
18:	lea

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 71460.3109651	total: 2.51ms	remaining: 2.51s
1:	learn: 68991.0387080	total: 4.46ms	remaining: 2.22s
2:	learn: 66647.1154003	total: 6.32ms	remaining: 2.1s
3:	learn: 64350.9110736	total: 8.15ms	remaining: 2.03s
4:	learn: 62192.0977510	total: 10.6ms	remaining: 2.1s
5:	learn: 60224.8079770	total: 13.1ms	remaining: 2.16s
6:	learn: 58298.8775427	total: 15.9ms	remaining: 2.26s
7:	learn: 56532.6279870	total: 18.3ms	remaining: 2.27s
8:	learn: 54686.0372942	total: 20.5ms	remaining: 2.25s
9:	learn: 53066.2711868	total: 22.8ms	remaining: 2.25s
10:	learn: 51456.3514504	total: 25.2ms	remaining: 2.26s
11:	learn: 49897.5543185	total: 27.9ms	remaining: 2.3s
12:	learn: 48453.5337521	total: 29.9ms	remaining: 2.27s
13:	learn: 47180.0696440	total: 32ms	remaining: 2.25s
14:	learn: 45925.6945196	total: 34.2ms	remaining: 2.25s
15:	learn: 44697.8646752	total: 36.5ms	remaining: 2.24s
16:	learn: 43571.6430576	total: 39.2ms	remaining: 2.26s
17:	learn: 42430.6126910	total: 41.6ms	remaining: 2.27s
18:	lea

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 70335.6320291	total: 2.9ms	remaining: 2.9s
1:	learn: 67981.8966061	total: 4.91ms	remaining: 2.45s
2:	learn: 65705.2226003	total: 6.63ms	remaining: 2.2s
3:	learn: 63580.3822739	total: 8.56ms	remaining: 2.13s
4:	learn: 61512.5220057	total: 11.8ms	remaining: 2.34s
5:	learn: 59601.2419926	total: 14.9ms	remaining: 2.46s
6:	learn: 57755.4244412	total: 17.2ms	remaining: 2.44s
7:	learn: 56048.4588818	total: 20ms	remaining: 2.48s
8:	learn: 54320.7053468	total: 22.2ms	remaining: 2.44s
9:	learn: 52742.2300640	total: 24.6ms	remaining: 2.43s
10:	learn: 51240.8437095	total: 27ms	remaining: 2.42s
11:	learn: 49833.1736889	total: 28.8ms	remaining: 2.37s
12:	learn: 48437.7689955	total: 30.9ms	remaining: 2.35s
13:	learn: 47104.1660034	total: 33.1ms	remaining: 2.33s
14:	learn: 45979.1055703	total: 35.9ms	remaining: 2.35s
15:	learn: 44749.9653623	total: 38ms	remaining: 2.34s
16:	learn: 43645.8487158	total: 40.3ms	remaining: 2.33s
17:	learn: 42608.0031776	total: 42.5ms	remaining: 2.32s
18:	learn: 

catboostのチューニングを行う

In [78]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)
rmse_scores = []
X_train['KFold'] = -1
#foldはvalidの対象となる値で０から４
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_train.loc[valid_index,'KFold'] = fold_id
    y_train.loc[valid_index,'KFold'] = fold_id

def objective(trial):
    #foldはvalidの対象となる値で０から４
    for fold in range(5):
        X_tr = X_train[X_train['KFold']!=fold].drop('KFold',axis=1)
        X_val = X_train[X_train['KFold']==fold].drop('KFold',axis=1)
        y_tr = y_train[y_train['KFold']!=fold].drop('KFold',axis=1)
        y_val = y_train[y_train['KFold']==fold].drop('KFold',axis=1)
    

        params = {
            'loss_function':'RMSE',
            'learning_rate':0.05,
            'iterations':5000,
            # 'calc_feature_importance': True,
            'use_best_model':True,
            # 'eval_metric':'rmse',
            'depth':trial.suggest_int('depth', 4, 10),
            'l2_leaf_reg':trial.suggest_float('l2_leaf_reg', 0.0000001, 0.0001, log=True),
            'random_strength' :trial.suggest_int('random_strength', 0, 100),
            'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
            'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
            'od_wait' :trial.suggest_int('od_wait', 10, 50)    
        }

        model = CatBoostRegressor(**params)
        model.fit(X_tr, y_tr, 
            cat_features=category_lists,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=100,
            plot=True
        )
        y_pred_val = model.predict(X_val)
        
        rmse_score = mean_squared_error(y_val, y_pred_val, squared=False)
    loss = np.mean(rmse_score)
    return loss

In [79]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)

[I 2022-09-15 19:02:26,221] A new study created in memory with name: no-name-321bfab4-da32-41e7-940b-e6c4b8ff1c84
/var/folders/qh/20ftf7rn5_58_rk3l25kt1nr0000gn/T/ipykernel_5468/4073598087.py:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 72762.7292810	test: 71490.9380778	best: 71490.9380778 (0)	total: 16.5ms	remaining: 1m 22s
1:	learn: 72124.8966704	test: 70901.6431514	best: 70901.6431514 (1)	total: 20.8ms	remaining: 51.9s
2:	learn: 71203.3322849	test: 69998.5100608	best: 69998.5100608 (2)	total: 31.1ms	remaining: 51.9s
3:	learn: 69389.4034257	test: 68395.4291333	best: 68395.4291333 (3)	total: 41.5ms	remaining: 51.8s
4:	learn: 67677.5815001	test: 66794.3370948	best: 66794.3370948 (4)	total: 48.4ms	remaining: 48.4s
5:	learn: 66478.9092343	test: 65675.7808461	best: 65675.7808461 (5)	total: 55.9ms	remaining: 46.5s
6:	learn: 65768.6652492	test: 65009.4570858	best: 65009.4570858 (6)	total: 66.6ms	remaining: 47.5s
7:	learn: 64271.7175414	test: 63672.1296879	best: 63672.1296879 (7)	total: 77.1ms	remaining: 48.1s
8:	learn: 62992.9283015	test: 62446.1536348	best: 62446.1536348 (8)	total: 83.4ms	remaining: 46.3s
9:	learn: 62312.3821251	test: 61788.5394338	best: 61788.5394338 (9)	total: 88.5ms	remaining: 44.2s
10:	learn

[W 2022-09-15 19:03:02,721] Trial 0 failed because of the following error: CatBoostError('y may be None only when X is an instance of catboost.Pool or string')
Traceback (most recent call last):
  File "/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/qh/20ftf7rn5_58_rk3l25kt1nr0000gn/T/ipykernel_5468/4073598087.py", line 40, in objective
    y_pred_val = model.fit(X_val)
  File "/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/catboost/core.py", line 5590, in fit
    return self._fit(X, y, cat_features, None, None, None, sample_weight, None, None, None, None, baseline,
  File "/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/catboost/core.py", line 2260, in _fit
    raise CatBoostError("y may be None only when X is an instance of catboost.Pool or string")
_catboost.CatBoostError: y may be None onl

CatBoostError: y may be None only when X is an instance of catboost.Pool or string

In [ ]:
study.best_params

チューニングしたパラメータを使ってみる

In [7]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index,:]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train.loc[train_index]
    y_val = y_train.loc[valid_index]

    model = CatBoostRegressor(
        loss_function='RMSE', 
        early_stopping_rounds=10,
        learning_rate=0.05, 
        depth=7, 
        l2_leaf_reg= 1.7538232373118053e-06, 
        random_strength=26, 
        bagging_temperature = 12.500712230836259,
        od_type='Iter',
        # od_wait = 10
        )

    model.fit(X_tr, y_tr, plot=True)

y_pred = model.predict(X_test)

CatBoostError: only one of the parameters od_wait, early_stopping_rounds should be initialized.

In [ ]:
df = pd.read_csv('../data/outer_data/test.csv')
id = df['id']
y_pred = pd.DataFrame(y_pred)
result = pd.concat([id, y_pred],axis=1)
result.to_csv('../data/result_data/CatBoost_cv_result.csv',index=False)